# 5월 IMC

## 임포트

In [1]:
import numpy as np # 넘파이
import matplotlib.pyplot as plt # 매트플롯립
import pandas as pd # 판다스(csv)
import re # 정규식 표현
from tqdm import tqdm
import os

import tensorflow as tf
import tensorflow_addons as tfa
import keras.backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
# 불러오기
train = pd.read_csv('/Users/minguinho/Documents/AI_Datasets/IMC_May_data/IMC_May_train.csv', encoding='cp949')
test = pd.read_csv('/Users/minguinho/Documents/AI_Datasets/IMC_May_data/IMC_May_test.csv', encoding='cp949')

### 트레이닝 데이터 975개, 테스트 데이터 107개가 있다

In [3]:
print("train : ", train.info())
print()
print("test : ", test.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Message_body  975 non-null    object
 1   Label         975 non-null    object
dtypes: object(2)
memory usage: 15.4+ KB
train :  None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Message_body  107 non-null    object
dtypes: object(1)
memory usage: 984.0+ bytes
test :  None


### dataframe에서 메일 제목과 라벨값을 추출해 데이터셋을 만들어준다

In [4]:
training_sentences = train['Message_body'].tolist()
testing_sentences = test['Message_body'].tolist()
training_temp = train['Label'].tolist()

training_labels = []
for i in range(0, 975) : 
    print(training_temp[i])
    if training_temp[i] == 'Non-Spam':
        print(training_temp[i])
        training_labels.append(0)
    else:
        training_labels.append(1)

training_labels = np.array(training_labels)

Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Spam
Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Spam
Non-Spam
Non-Spam
Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spam
Non-Spa

In [5]:
vocab_size = 10000 # 단어 크기
embedding_dim = 16 # 임패딩 단위
max_length = 128 # 리뷰에 들어있는 최대 단어 갯수. 아무리 길어도 단어 120개가 한계다...라는 것을 나타냄.
trunc_type='post' # truncation type. 단어 갯수가 120개보다 많을 때 앞에서부터 자를건지 뒤에서부터 자를건지 결정함 post는 문자열이 120개보다 많으면 뒤에서 자르겠다는 뜻.
oov_tok = "<OOV>" # 토큰 딕셔너리에 없는 단어는 OOV로 표시

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok) # num_words = vocab_size : 가장 많이 쓰이는 단어 'vocab_size'개만 토큰화 하는거. 그러니까 토큰화 대상인 문자여렝 단어가 20000개 있으면 많이 쓰이는 순으로 나열한 뒤 앞에 있는 10000개의 단어만 토큰화 하고 나머지 10000개는 토큰화하지 않는다. 얘들은 뭐 OOV로 나오겠지
tokenizer.fit_on_texts(training_sentences) # 토큰화
word_index = tokenizer.word_index # 토큰화 결과 가져오기
sequences = tokenizer.texts_to_sequences(training_sentences) # 문장들을 토큰 순서로 나열하기
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type) # 패딩. 

testing_sequences = tokenizer.texts_to_sequences(testing_sentences) # 훈련셋으로 만든 토크나이저로 테스트셋의 시퀀스 출력. 토큰화 하지않은 단어가 많기 때문에 기존에 만들어놨던 걸로 한다. 만약 토크나이저를 새로 만든다면 두 토크나이저는 다른 토크나이저가 되기 때문에 원하는 결과를 얻을 수 없다. 
testing_padded = pad_sequences(testing_sequences,maxlen=max_length) # 패딩


## 모델 생성

In [6]:
EPOCH = 200
BATCH_SIZE = 128

filename = 'checkpoint_bestModel.h5'.format(EPOCH, BATCH_SIZE)
checkpoint = ModelCheckpoint(filename,             # 파일 이름
                             monitor = 'val_f1_score',   # val_loss 값이 개선되면 개선 모델을 filename으로 저장
                             verbose=1,            # 로그 출력
                             save_best_only=True,  # 가장 최고의 val_loss를 보이는 모델만 저장
                             mode='max'           
                            )

# 모델 생성
model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[None]),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)), # 첫번째 LSTM. return_sequences=True를 해줘야한다. 두번 째 LSTM에 모든 시퀀스(아웃풋)을 다음 LSTM에 넣어야한다. 
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)), # 두번째 LSTM
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=[tfa.metrics.F1Score(num_classes=2, average='micro',threshold=0.5)]) # 컴파일. 긍정, 부정밖에 없으니 이진 분류를 하니까 binary_crossentropy를 손실 함수로 사용한다. 

In [7]:
model.fit(padded, training_labels, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split=0.2, callbacks=[checkpoint]) # 훈련

Epoch 1/200
7/7 [==============================] - 10s 499ms/step - loss: 0.5448 - f1_score: 0.0000e+00 - val_loss: 0.6766 - val_f1_score: 0.0000e+00

Epoch 00001: val_f1_score improved from -inf to 0.00000, saving model to checkpoint_bestModel.h5
Epoch 2/200
7/7 [==============================] - 1s 148ms/step - loss: 0.4247 - f1_score: 0.0000e+00 - val_loss: 0.8143 - val_f1_score: 0.0000e+00

Epoch 00002: val_f1_score did not improve from 0.00000
Epoch 3/200
7/7 [==============================] - 1s 145ms/step - loss: 0.4061 - f1_score: 0.0000e+00 - val_loss: 0.8412 - val_f1_score: 0.0000e+00

Epoch 00003: val_f1_score did not improve from 0.00000
Epoch 4/200
7/7 [==============================] - 1s 145ms/step - loss: 0.4192 - f1_score: 0.0000e+00 - val_loss: 0.7447 - val_f1_score: 0.0000e+00

Epoch 00004: val_f1_score did not improve from 0.00000
Epoch 5/200
7/7 [==============================] - 1s 140ms/step - loss: 0.3644 - f1_score: 0.0000e+00 - val_loss: 0.7018 - val_f1_score:

## 모델 예측
### 가장 정확도가 높은 모델을 사용 후 예측해준다

In [10]:
model.load_weights(filename)

prediction = model.predict(testing_padded)

### 출력값이 0~1 사이의 실수로 되어있으니 이를 0.5 기준으로 'Spam', 'Non-Spam'으로 나눠준다

In [12]:
pred_final = []

for i in range(0, len(prediction)) :
    if prediction[i] > 0.5 :
        pred_final.append(1)
    else:
        pred_final.append(0)

pred_final = np.asarray(pred_final)

### csv파일에 저장해준다

In [13]:
predict_toCsv = {"Prediction": pred_final}

submission = pd.DataFrame()

submission['Prediction'] = predict_toCsv["Prediction"]

In [14]:
submission.to_csv('김민규_Adv_result.csv', index=False) # 파일 저장